In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Declare a base
base = automap_base()
# Use base to reflect the database tables 
base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
# Print out classes mapped on to base 
base.classes.keys()

In [ ]:
# Save references to each table
# Reflect tables into classes and save a reference to those classes called Station and Measurement
station = base.classes.station
measurement = base.classes.measurement


In [ ]:
# Create our session (link) from Python to the DB
##Link Python to the database by creating an SQLAlchemy session.
session = Session(engine)


In [67]:
# Understand table and types of data
inspector = inspect(engine)
inspector.get_table_names()

columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c['type'])

columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Exploratory Precipitation Analysis

In [68]:
# Find the most recent date in the data set.
most_recent = session.query(measurement.date).order_by(measurement.date.desc()).first()
most_recent

('2017-08-23')

In [107]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
query = Session.query

# Save the query results as a Pandas DataFrame and set the index to the date column
results = session.query(measurement.date, measurement.prcp).\
    filter(measurement.date >= one_year)

# Queries into Dataframe 
precip_df = pd.DataFrame(results).set_index('date')
precip_df = precip_df.sort_values(by='date').dropna()
precip_df.head()

,prcp
date,
2016-08-23,0.00
2016-08-23,1.79
2016-08-23,0.05
2016-08-23,0.15
2016-08-23,0.70


In [106]:
# Use Pandas Plotting with Matplotlib to plot the data
'''
x = precip_df['date']
y = precip_df['prcp']

plt.bar(x, y, color="blue", align="center")
plt.bar(precip_df['date'], precip_df['prcp'], color="blue", align="center")
plt.show()

precip_df.plot('date', 'prcp')
plt.title("Precipitation Analysis")
plt.xlabel("Date: August 23, 2016 - August 23, 2017")
plt.ylabel("Precipitaiton")
'''

KeyError: 'date'

In [103]:
# Use Pandas to calculate the summary statistics for the precipitation data
precip_df.describe()

,prcp
count,2021.000000
mean,0.177279
std,0.461190
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


# Exploratory Station Analysis

In [111]:
# Design a query to calculate the total number stations in the dataset
station_total = session.query(station).count()
station_total

9

In [112]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
most_rows = session.query(measurement.station, func.count(measurement.station)).\
        
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()